In [2]:
import warnings
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns

from lightgbm import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.base import clone

from doubleml import DoubleMLData
from doubleml import DoubleMLPLR

In [3]:
import pandas as pd
import doubleml

In [53]:
df = pd.read_csv("causal_challenge_limited_data.csv")

In [54]:
np.random.seed(42)

from sklearn.linear_model import LogisticRegression, LinearRegression, LogisticRegressionCV, RidgeClassifierCV, PassiveAggressiveClassifier
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from xgboost import XGBClassifier, XGBRegressor

rf_g = RandomForestRegressor(n_estimators = 500, max_depth = 15, max_features = 4, min_samples_leaf = 7)
ml_g = XGBRegressor(tree_method="hist", objective = "reg:squarederror", eta = 0.01, n_estimators =50)
rf_m = RandomForestClassifier(n_estimators = 500, max_depth = 5, max_features = 4, min_samples_leaf = 7)
ml_m = LogisticRegressionCV(cv=5) 


dml_data = doubleml.DoubleMLData(df, y_col = 'Y', d_cols = 'treat_binary', x_cols=['x1', 'x2'])

dml_irm = doubleml.DoubleMLIRM(dml_data, ml_g=ml_g, ml_m=ml_m)
dml_irm.fit(store_predictions=True)

print(dml_irm.summary)

                   coef  std err           t  P>|t|      2.5 %     97.5 %
treat_binary  50.764243  0.38893  130.522884    0.0  50.001954  51.526531


Despite this being (I think) the same set up as the R code, XGB regressor for G-estimation and LogisticRegression for the M, the results are way off

In [55]:
dml_plr = doubleml.DoubleMLPLR(dml_data, ml_l=rf_g, ml_m=rf_m)
dml_plr.fit(store_predictions=True)

print(dml_plr.summary)

                  coef   std err        t  P>|t|      2.5 %     97.5 %
treat_binary  18.86101  0.393334  47.9516    0.0  18.090089  19.631932


PartialLR doesn't do any better.